In [1]:
import pandas as pd 
import numpy as np

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [24]:
aggregate = [
  {
    "$match": {
      "data_description.project_name": "V1 Deep Dive"
    }
  },
  {
    "$project": {
      "name": 1, 
      "subject_id": "$data_description.subject_id",
      "genotype": "$subject.subject_details.genotype", 
      "date_of_birth": "$subject.subject_details.date_of_birth", 
      "sex": "$subject.subject_details.sex", 
      "session_type": "$acquisition.acquisition_type",
      "session_time": "$acquisition.acquisition_start_time",
      "stimulus_epochs": "$acquisition.stimulus_epochs.stimulus_name",  
      "project_name": "$data_description.project_name", 
      "modality": "$data_description.modalities.name",
      "column": { "$arrayElemAt": ["$data_description.tags", 0] },
      "volume": { "$arrayElemAt": ["$data_description.tags", 1] }
    }
  }
]
    
records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [25]:
df = pd.DataFrame(records)
df.head()

,_id,name,subject_id,genotype,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,column,volume
0,3b85c659-20c8-438f-ab58-de1aac3b81cf,416296_2018-11-29_12-08-40_nwb_2025-08-08_16-2...,416296,Camk2a-tTA/wt;tetO-GCaMP6s/wt,2018-08-06,Female,V1DD Session,2018-11-29 12:08:40.014190-08:00,"[drifting_gratings_volume_full, drifting_grati...",V1 Deep Dive,"[Planar optical physiology, Behavior videos]",Column 2,Volume 5
1,b98ce4a9-a66b-4b70-baa6-95ef451ae087,427836_2019-04-25_12-16-58_nwb_2025-08-08_16-2...,427836,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,V1DD Session,2019-04-25 12:16:58.240890-07:00,"[drifting_gratings_volume_full, drifting_grati...",V1 Deep Dive,"[Planar optical physiology, Behavior videos]",Column 5,Volume 3
2,5cccd09c-4ae8-4d8e-ae92-23099d22bbe2,427836_2019-04-24_13-06-45_nwb_2025-08-08_16-2...,427836,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,V1DD Session,2019-04-24 13:06:45.257460-07:00,"[drifting_gratings_volume_full, drifting_grati...",V1 Deep Dive,"[Planar optical physiology, Behavior videos]",Column 4,Volume 3
3,911d215f-dafa-4b39-86f7-b36336974da1,427836_2019-04-25_13-49-39_nwb_2025-08-08_16-2...,427836,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,V1DD Session,2019-04-25 13:49:39.163550-07:00,"[drifting_gratings_volume_full, drifting_grati...",V1 Deep Dive,"[Planar optical physiology, Behavior videos]",Column 5,Volume 4
4,c918f874-d534-4341-b0dc-f1aba388af66,427836_2019-04-26_12-54-40_nwb_2025-08-08_16-2...,427836,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,2018-10-08,Female,V1DD Session,2019-04-26 12:54:40.332660-07:00,"[drifting_gratings_volume_full, drifting_grati...",V1 Deep Dive,"[Planar optical physiology, Behavior videos]",Column 2,Volume 4


In [26]:
df.to_csv('/data/metadata/V1DD_metadata.csv', index= False)